# سیمینٹک کرنل کے ساتھ اوپن بی این بی ایم سی پی سرور کا انضمام

یہ نوٹ بک دکھاتی ہے کہ سیمینٹک کرنل کو اصل اوپن بی این بی ایم سی پی سرور کے ساتھ استعمال کرتے ہوئے ایم سی پی اسٹوڈیو پلگ ان کے ذریعے حقیقی ایئر بی این بی رہائش گاہوں کی تلاش کیسے کی جا سکتی ہے۔ ایل ایل ایم تک رسائی کے لیے، یہ ایزور اے آئی فاؤنڈری کا استعمال کرتی ہے۔ اپنے ماحول کے متغیرات کو ترتیب دینے کے لیے، آپ [سیٹ اپ سبق](/00-course-setup/README.md) کی پیروی کر سکتے ہیں۔


## ضروری پیکجز درآمد کریں


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## ایم سی پی پلگ ان کنکشن بنانا

ہم [OpenBnB MCP سرور](https://github.com/openbnb-org/mcp-server-airbnb) سے MCPStdioPlugin کے ذریعے جڑیں گے۔ یہ سرور @openbnb/mcp-server-airbnb پیکج کے ذریعے ایئر بی این بی تلاش کی سہولت فراہم کرتا ہے۔


## کلائنٹ بنانا

اس مثال میں، ہم Azure AI Foundry کو اپنے LLM تک رسائی کے لیے استعمال کریں گے۔ یقینی بنائیں کہ آپ کے ماحول کے متغیرات درست طریقے سے ترتیب دیے گئے ہیں۔


## ماحول کی تشکیل

Azure OpenAI کی ترتیبات کو ترتیب دیں۔ یقینی بنائیں کہ آپ کے پاس درج ذیل ماحول کے متغیرات سیٹ کیے گئے ہیں:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## اوپن بی این بی ایم سی پی انٹیگریشن کو سمجھنا

یہ نوٹ بک **حقیقی اوپن بی این بی ایم سی پی سرور** سے جڑتی ہے جو اصل ایئربی این بی سرچ کی سہولت فراہم کرتا ہے۔

### یہ کیسے کام کرتا ہے:

1. **MCPStdioPlugin**: ایم سی پی سرور کے ساتھ معیاری ان پٹ/آؤٹ پٹ کمیونیکیشن استعمال کرتا ہے
2. **حقیقی این پی ایم پیکیج**: `@openbnb/mcp-server-airbnb` کو npx کے ذریعے ڈاؤنلوڈ اور چلایا جاتا ہے
3. **لائیو ڈیٹا**: ایئربی این بی کے APIs سے اصل پراپرٹی ڈیٹا واپس کرتا ہے
4. **فنکشن دریافت**: ایجنٹ خود بخود ایم سی پی سرور سے دستیاب فنکشنز کو دریافت کرتا ہے

### دستیاب فنکشنز:

اوپن بی این بی ایم سی پی سرور عام طور پر فراہم کرتا ہے:
- **search_listings** - مقام اور معیار کے مطابق ایئربی این بی پراپرٹیز کی تلاش
- **get_listing_details** - مخصوص پراپرٹیز کی تفصیلی معلومات حاصل کریں
- **check_availability** - مخصوص تاریخوں کے لیے دستیابی چیک کریں
- **get_reviews** - پراپرٹیز کے لیے ریویوز حاصل کریں
- **get_host_info** - پراپرٹی کے میزبانوں کی معلومات حاصل کریں

### ضروریات:

- آپ کے سسٹم پر **Node.js** انسٹال ہو
- ایم سی پی سرور پیکیج ڈاؤنلوڈ کرنے کے لیے **انٹرنیٹ کنکشن**
- **NPX** دستیاب ہو (یہ Node.js کے ساتھ آتا ہے)

### کنکشن کی جانچ:

آپ ایم سی پی سرور کو دستی طور پر چلانے کے لیے یہ کمانڈ استعمال کر سکتے ہیں:
```bash
npx -y @openbnb/mcp-server-airbnb
```

یہ اوپن بی این بی ایم سی پی سرور کو ڈاؤنلوڈ اور شروع کرے گا، جس سے سیمینٹک کرنل حقیقی ایئربی این بی ڈیٹا کے لیے جڑتا ہے۔


## اوپن بی این بی ایم سی پی سرور کے ساتھ ایجنٹ چلانا

اب ہم اے آئی ایجنٹ کو چلائیں گے جو اوپن بی این بی ایم سی پی سرور سے جڑتا ہے تاکہ اسٹاک ہوم میں 2 بالغوں اور 1 بچے کے لیے حقیقی ایئر بی این بی رہائش تلاش کی جا سکے۔ تلاش کے معیار کو تبدیل کرنے کے لیے `user_inputs` فہرست کو تبدیل کرنے میں آزاد محسوس کریں۔


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# خلاصہ  
مبارک ہو! آپ نے کامیابی سے ایک AI ایجنٹ تیار کیا ہے جو حقیقی دنیا کے رہائش کی تلاش کے ساتھ Model Context Protocol (MCP) کے ذریعے مربوط ہوتا ہے:

## استعمال شدہ ٹیکنالوجیز:  
- Semantic Kernel - Azure OpenAI کے ساتھ ذہین ایجنٹس بنانے کے لیے  
- Azure AI Foundry - LLM صلاحیتوں اور چیٹ مکمل کرنے کے لیے  
- MCP (Model Context Protocol) - معیاری ٹول انضمام کے لیے  
- OpenBnB MCP Server - حقیقی Airbnb تلاش کی فعالیت کے لیے  
- Node.js/NPX - بیرونی MCP سرور چلانے کے لیے  

## آپ نے کیا سیکھا:  
- MCP انضمام: Semantic Kernel ایجنٹس کو بیرونی MCP سرورز سے جوڑنا  
- حقیقی وقت میں ڈیٹا تک رسائی: لائیو APIs کے ذریعے حقیقی Airbnb پراپرٹیز کی تلاش  
- پروٹوکول مواصلات: ایجنٹ اور MCP سرور کے درمیان stdio مواصلات کا استعمال  
- فنکشن دریافت: MCP سرورز سے دستیاب فنکشنز کو خودکار طور پر دریافت کرنا  
- اسٹریمنگ جوابات: حقیقی وقت میں فنکشن کالز کو کیپچر اور لاگ کرنا  
- HTML رینڈرنگ: ایجنٹ کے جوابات کو اسٹائل شدہ ٹیبلز اور انٹرایکٹو ڈسپلے کے ساتھ فارمیٹ کرنا  

## اگلے اقدامات:  
- اضافی MCP سرورز کو مربوط کریں (موسم، پروازیں، ریستوران)  
- MCP اور A2A پروٹوکولز کو یکجا کرتے ہوئے ایک ملٹی ایجنٹ سسٹم بنائیں  
- اپنے ڈیٹا ذرائع کے لیے حسب ضرورت MCP سرورز بنائیں  
- سیشنز کے دوران مستقل گفتگو کی یادداشت نافذ کریں  
- MCP سرور آرکیسٹریشن کے ساتھ ایجنٹ کو Azure Functions پر تعینات کریں  
- صارف کی تصدیق اور بکنگ کی صلاحیتیں شامل کریں  



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہیں۔ اصل دستاویز، جو اس کی اصل زبان میں ہے، کو مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
